In [1]:
import pandas as pd
import os
import numpy as np
from PyPDF2 import PdfReader
import re
import sys
from operator import itemgetter
import fitz
import json
import string

Root dir for data

In [2]:
raw_data_path = "raw_data"

In [3]:
files = []
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for filename in filenames: 
        files.append(os.path.join(dirpath, filename))

print(files)

['raw_data/.DS_Store', 'raw_data/Guide_Art_1_ENG.pdf']


In [4]:
pdfs = []
for file in files:
    if "pdf" in file:
        pdfs.append({"pdf": PdfReader(file),"path": file})
        
print(pdfs)

[{'pdf': <PyPDF2._reader.PdfReader object at 0x123b88510>, 'path': 'raw_data/Guide_Art_1_ENG.pdf'}]


Check one PDF

In [5]:
pdf = pdfs[0]
pages = pdf['pdf'].pages
print(pages[0].extract_text())
print(len(pages))

 
 
 
Guide on A rticle  1 
of the  European  Convention  
on Human Rights  
Obligation to respect  human  rights  – 
Concepts of “jurisdiction ” 
and imputability  
Updated on 31 August  2023 
 
 
 
 
 
 
 
 
 
 
Prepared by the Registry. It does not bind the Court . 
  
53


In [6]:
sections = []
#for pdf in pdfs:
pdf = pdfs[0]
print("The case document path: ", pdf['path'])
print("The # of page in this document:", len(pdf['pdf'].pages))
for index, page in enumerate(pdf['pdf'].pages):
    text = page.extract_text()
    p = re.compile("\n\d+\.  ")
    for m in p.finditer(text):
        # print(m)
        l = [int(s) for s in re.findall("\d+", m.group())]
        print(l)
        if l is not None:
            sections.append({'number': l[0], 'start': m.start(), 'page_num': index, 'text': text})

The case document path:  raw_data/Guide_Art_1_ENG.pdf
The # of page in this document: 53
[1]
[2]
[3]
[4]
[5]
[1]
[2]
[3]
[1]
[2]
[1]
[2]
[1]
[1]
[2]
[3]
[5]
[6]
[7]
[8]
[9]
[10]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[1]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[3]
[32]
[33]
[34]
[35]
[4]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[5]
[49]
[50]
[51]
[52]
[53]
[55]
[56]
[57]
[1]
[58]
[59]
[60]
[61]
[62]
[63]
[2]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[3]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[1]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[2]
[139]
[140]
[141]
[142]
[143]
[1]
[144]
[2]
[145]
[146]
[147]
[1]
[148]
[149]
[2]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]

In [10]:
# with open("text.txt","w+") as file:
#     for section in sections:
#         file.write(str(section))

df = pd.DataFrame(sections)
print(df.columns)
df = df[df['page_num'] >= 4 ]
df['text'][55]
df['end'] = df.groupby('page_num')['start'].shift(-1)
df=df.dropna()
df

Index(['number', 'start', 'page_num', 'text'], dtype='object')


,number,start,page_num,text,end
13,1,532,5,Guide on Article 1 of the Convention – Obligat...,1199.0
14,2,1199,5,Guide on Article 1 of the Convention – Obligat...,2732.0
16,5,765,6,Guide on Article 1 of the Convention – Obligat...,1166.0
17,6,1166,6,Guide on Article 1 of the Convention – Obligat...,1392.0
18,7,1392,6,Guide on Article 1 of the Convention – Obligat...,2371.0
...,...,...,...,...,...
172,152,2483,46,Guide on Article 1 of the Convention – Obligat...,3096.0
173,153,3096,46,Guide on Article 1 of the Convention – Obligat...,4288.0
175,155,330,47,Guide on Article 1 of the Convention – Obligat...,992.0
176,156,992,47,Guide on Article 1 of the Convention – Obligat...,1845.0
